In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from keras import layers
import sys
sys.path.insert(1, "drive/MyDrive/Informer-Tensorflow/models")
sys.path.insert(1, "drive/MyDrive/Informer-Tensorflow")

In [3]:
from attn import ProbAttention, AttentionLayer

In [4]:
class MultiHeadProbAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadProbAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.attn = AttentionLayer(ProbAttention(False), self.d_model, self.num_heads)
    def call(self, inputs):
        query = inputs
        key = inputs
        value = inputs
        x = self.attn([query, key, value])
        return x


In [5]:
def positional_encoding(length, depth):
    depth = depth/2

    positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
    depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

    angle_rates = 1 / (10000**depths)         # (1, depth)
    angle_rads = positions * angle_rates      # (pos, depth)

    pos_encoding = np.concatenate(
        [np.sin(angle_rads), np.cos(angle_rads)],
        axis=-1)

    return tf.cast(pos_encoding, dtype=tf.float32)

class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, d_model):
        super().__init__()
        self.d_model = d_model
        # self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True)
        self.pos_encoding = positional_encoding(length=2048, depth=d_model)

    # def compute_mask(self, *args, **kwargs):
    #     return self.embedding.compute_mask(*args, **kwargs)

    def call(self, x):
        length = tf.shape(x)[1]
        # x = self.embedding(x)
        # This factor sets the relative scale of the embedding and positonal_encoding.
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x = x + self.pos_encoding[tf.newaxis, :length, :]
        return x

In [6]:
class ConvLayer(keras.layers.Layer):
    def __init__(self, c_in):
        super(ConvLayer, self).__init__()
        self.downConv = tf.keras.layers.Conv1D(
                                  filters=c_in,
                                  kernel_size=3,
                                  padding='causal')
        self.norm = tf.keras.layers.BatchNormalization()
        self.activation = tf.keras.layers.ELU()
        self.maxPool = tf.keras.layers.MaxPool1D(pool_size=3)#, strides=2)

    def call(self, x, **kargs):
        x = self.downConv(x)
        x = self.norm(x)
        x = self.activation(x)
        x = self.maxPool(x)
        return x

In [7]:
BATCH_SIZE = 32
def models():
    inputs = keras.layers.Input(shape=(127,15), batch_size=BATCH_SIZE)
    # x = keras.layers.Dense(32, activation="tanh")(inputs)
    x = keras.layers.Conv1D(32, 1, activation='tanh')(inputs)
    x = PositionalEmbedding(32)(x)
    x = MultiHeadProbAttention(128, 32)(x)
    x = ConvLayer(64)(x)
    x = MultiHeadProbAttention(64, 32)(x)
    x = ConvLayer(32)(x)
    x = keras.layers.LayerNormalization()(x)
    x = keras.layers.GlobalAveragePooling1D(data_format='channels_first')(x)
    x = keras.layers.Dense(5, activation='softmax')(x)
    return keras.Model(inputs, x)

In [8]:
model = models()
model.summary()
# del model

[TensorShape([32, 127, 32]), TensorShape([32, 127, 32]), TensorShape([32, 127, 32])]
[TensorShape([32, 42, 64]), TensorShape([32, 42, 64]), TensorShape([32, 42, 64])]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(32, 127, 15)]           0         
                                                                 
 conv1d (Conv1D)             (32, 127, 32)             512       
                                                                 
 positional_embedding (Posit  (32, 127, 32)            0         
 ionalEmbedding)                                                 
                                                                 
 multi_head_prob_attention (  (32, 127, 128)           29184     
 MultiHeadProbAttention)                                         
                                                                 
 conv_layer (ConvLayer)   

In [9]:
x = tf.random.normal((BATCH_SIZE, 127, 15))
model(x)

<tf.Tensor: shape=(32, 5), dtype=float32, numpy=
array([[0.19999997, 0.19999999, 0.19999999, 0.20000002, 0.20000002],
       [0.20000002, 0.19999999, 0.19999999, 0.19999999, 0.20000002],
       [0.20000002, 0.19999999, 0.19999997, 0.19999996, 0.20000002],
       [0.19999994, 0.19999999, 0.20000002, 0.20000003, 0.20000002],
       [0.2       , 0.19999996, 0.2       , 0.2       , 0.2       ],
       [0.19999999, 0.20000002, 0.19999999, 0.20000002, 0.19999999],
       [0.19999997, 0.2       , 0.2       , 0.2       , 0.19999997],
       [0.19999999, 0.19999997, 0.19999999, 0.20000002, 0.20000002],
       [0.19999997, 0.19999999, 0.20000002, 0.20000002, 0.19999999],
       [0.20000002, 0.20000002, 0.20000002, 0.19999997, 0.19999999],
       [0.19999999, 0.19999996, 0.19999999, 0.20000002, 0.20000002],
       [0.20000002, 0.19999997, 0.20000002, 0.19999997, 0.19999999],
       [0.19999997, 0.20000002, 0.19999999, 0.20000003, 0.20000003],
       [0.2       , 0.2       , 0.2       , 0.19999997

In [11]:
data = np.load("mfcc.npz")
X = data["X"]
Y = data["Y"]

x_train = X[0:832]
y_train = Y[0:832]
x_test = X[872:]
y_test = Y[872:]

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=100000).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
val_dataset = val_dataset.shuffle(buffer_size=1000).batch(BATCH_SIZE)

In [13]:
x = x_train[0:BATCH_SIZE]
x_rank = tf.rank(x).numpy()
x_norm_resize_shape = [BATCH_SIZE] + list(tf.ones(tf.rank(x), dtype=tf.int32).numpy())[1:]

In [14]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super().__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, dtype=tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [15]:
def perturbation_loss(x, y, from_logits=False):
    return keras.losses.CategoricalCrossentropy(from_logits=from_logits)(x, y)

loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
learning_rate = CustomSchedule(32)
optimizer = keras.optimizers.experimental.AdamW(learning_rate=learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

In [16]:
eps = 4.     # the perturbation parameter
sig = 1e-5   # initial perturbation StdDev
zeta = 1e-6  # differentiation constant
lamd = 1     # regularization parameter

@tf.function
def training_step(x, label):
    x_p = tf.random.normal(x.shape, stddev=sig)
    x_norm = x_p
    for i in range(x_rank-1, 0, -1):
        x_norm = tf.norm(x_norm, ord=2, axis=int(i))
    x_p /= tf.reshape(x_norm, (BATCH_SIZE, 1, 1))
    x_p *= zeta


    with tf.GradientTape() as model_tape, tf.GradientTape() as perturbation_tape:
        perturbation_tape.watch(x_p)
        y_p = model(x + x_p, training=True)
        y = model(x, training=True)
        l = perturbation_loss(y, y_p)
        g = perturbation_tape.gradient(l, x_p)

        g_norm = g
        for i in range(x_rank-1, 0, -1):
            g_norm = tf.norm(g_norm, ord=2, axis=int(i))

        x_p = eps * g / tf.reshape(g_norm, x_norm_resize_shape)

        y_p = model(x + x_p, training=True)
        y = model(x, training=True)
        l = perturbation_loss(y, y_p)    # Recalculate regularization

        logits = model(x, training=True)
        loss = loss_fn(label, logits) + lamd * l / BATCH_SIZE
        grads = model_tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

    train_acc_metric.update_state(label, logits)
    return loss, train_acc_metric.result(), l, x_p, g

In [17]:
train_loss = []
train_metric = []
val_metric = []
p_loss = []
for epoch in tqdm(range(1000)):
    print("\nStart of epoch %d" % (epoch,))
    for step, (x, label) in enumerate(train_dataset):
        loss, train_acc, l, x_p, g = training_step(x, label)

    print(
        "Training loss: %.4f\nTraining metric: %.4f"
        % (float(loss), float(train_acc))
    )
    print("perturbation loss: %.4f" % float(l))

    for x_batch_val, y_batch_val in val_dataset:
        val_logits = model(x_batch_val, training=False)
        # Update val metrics
        val_acc_metric.update_state(y_batch_val, val_logits)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc)))

    train_loss.append(loss)
    train_metric.append(train_acc)
    val_metric.append(val_acc)
    p_loss.append(l)

    tl = np.array(train_loss)
    tm = np.array(train_metric)
    vm = np.array(val_metric)
    pl = np.array(p_loss)

    np.savez("logs.npz", train_loss=tl, train_acc=tm, val_acc=vm, p_loss=pl, x_p=x_p)

  0%|          | 0/1000 [00:00<?, ?it/s]


Start of epoch 0
Training loss: 1.6597
Training metric: 0.1995
perturbation loss: 1.6094


  0%|          | 1/1000 [00:23<6:30:30, 23.45s/it]

Validation acc: 0.2891

Start of epoch 1
Training loss: 1.6597
Training metric: 0.1995
perturbation loss: 1.6094


  0%|          | 2/1000 [00:25<2:55:46, 10.57s/it]

...

Validation acc: 0.9766

Start of epoch 998
Training loss: 0.0000
Training metric: 0.9878
perturbation loss: 0.0003


100%|█████████▉| 999/1000 [29:04<00:01,  1.71s/it]

Validation acc: 0.9766

Start of epoch 999
Training loss: 0.0000
Training metric: 0.9878
perturbation loss: 0.0003


100%|██████████| 1000/1000 [29:05<00:00,  1.75s/it]

Validation acc: 0.9688
